# AlphaCare Insurance Solutions (ACIS) 


AlphaCare Insurance Solutions (ACIS) is committed to developing cutting-edge risk and predictive analytics in the area of car insurance
planning and marketing in South Africa.This is a part of the final report on hypothesis testing,

#### Outline
1. Null hypothesis to be tested
2. Identifying KPIs
3. Data Segmentation
4. Analyze the hypothesis

In [2]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add the parent directory to the Python path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from scripts.data_loader import load_data

# Load the text file into a Pandas DataFrame
source = pd.read_csv('../data/data.txt', delimiter='|')  

source.to_csv('../data/data2.csv', index=False)
df= pd.read_csv('../data/data2.csv')
df.head()


C:\Users\bethelhem.teka\AppData\Local\Temp\ipykernel_32280\3708453824.py:14: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  source = pd.read_csv('../data/data.txt', delimiter='|')
C:\Users\bethelhem.teka\AppData\Local\Temp\ipykernel_32280\3708453824.py:17: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('../data/data2.csv')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


## Null Hypotesis to be tested 

There are no risk differences across provinces, There are no risk differences between zip codes, There are no significant margin (profit) difference between zip codes, and There are not significant risk difference between Women and Men


## Identify KPIs 

In [3]:
# Loss ratio by province
df['LossRatio']  = df['TotalClaims']/ df[df['TotalPremium'] >0]['TotalPremium'] 
province_loss_ratio = df.groupby('Province')['LossRatio'].mean()
province_loss_ratio

Province
Eastern Cape     0.235636
Free State       0.106209
Gauteng          0.428927
KwaZulu-Natal    0.265328
Limpopo          0.348712
Mpumalanga       0.392698
North West       0.285348
Northern Cape    0.203831
Western Cape     0.341810
Name: LossRatio, dtype: float64

In [4]:
# Loss ratio by postal code
postalcode_loss_ratio = df.groupby('PostalCode')['LossRatio'].mean()
postalcode_loss_ratio.head(10)

PostalCode
1     0.330662
2     0.852758
4     0.000000
5     0.698901
6     0.122927
7     0.000000
8     0.046875
17    0.000000
21    0.493564
22    0.000000
Name: LossRatio, dtype: float64

In [5]:
# Profit margin by postal code
df['ProfitMargin']  = df['TotalPremium'] - df['TotalClaims']/ df[df['TotalPremium'] >0]['TotalPremium']
postal_code_profit_margin = df.groupby('PostalCode')['ProfitMargin'].mean()
postal_code_profit_margin

PostalCode
1        95.538484
2        92.350502
4       113.947737
5        67.805129
6       101.059937
           ...    
9781     81.496622
9830    131.760895
9868    116.042377
9869     74.856604
9870     80.469292
Name: ProfitMargin, Length: 888, dtype: float64

In [6]:
# Loss ratio by postal code by gender
postalcode_loss_ratio = df.groupby('Gender')['LossRatio'].mean()
postalcode_loss_ratio.head(10)

Gender
Female           0.491979
Male             0.353258
Not specified    0.347932
Name: LossRatio, dtype: float64

## Data Segementations

In [7]:
feature_to_test = 'CoverType'


# Create two groups

group_a = df[df[feature_to_test].isin(['Windscreen', 'Own Damage' ,'Third Party', 'Passenger Liability', 'Signage and Vehicle Wraps', 'Keys and Alarms', 
'Cleaning and Removal of Accident Debris' ,'Emergency Charges', 'Credit Protection' ,'Roadside Assistance', 'Accidental Death' ])]
group_b = df[df[feature_to_test].isin(['Basic Excess Waiver' ,'Income Protector' ,'Deposit Cover',  'Standalone passenger liability' ,'Baggage/Luggage',
 'Asset Value Preserver', 'Cash Takings', 'Third Party Only' 'Trailer',  'Fire and Theft' ,'Factory Fitted Sound aner Electronic Equipment'])]

group_a = group_a[group_a['TotalPremium']>0]
group_b = group_b[group_b['TotalPremium']>0]


In [8]:
from scipy.stats import ttest_ind, chi2_contingency
numerical_columns = ['SumInsured', 'CalculatedPremiumPerTerm', 'CustomValueEstimate']

for col in numerical_columns:
    t_stat, p_val = ttest_ind(group_a[col].dropna(), group_b[col].dropna())
    print(f"{col}: T-statistic = {t_stat:.2f}, P-value = {p_val:.3f}")
    if p_val > 0.05:
        print(f"  -> No significant difference (groups are equivalent for {col})")
    else:
        print(f"  -> Significant difference (adjust groups for {col})")

SumInsured: T-statistic = 113.20, P-value = 0.000
  -> Significant difference (adjust groups for SumInsured)
CalculatedPremiumPerTerm: T-statistic = 25.09, P-value = 0.000
  -> Significant difference (adjust groups for CalculatedPremiumPerTerm)
CustomValueEstimate: T-statistic = -0.60, P-value = 0.549
  -> No significant difference (groups are equivalent for CustomValueEstimate)


In [9]:
categorical_columns = ['Gender', 'VehicleType', 'MaritalStatus']

for col in categorical_columns:
    contingency_table = pd.crosstab(df[col], df[feature_to_test])
    chi2, p_val, _, _ = chi2_contingency(contingency_table)
    print(f"{col}: Chi-square = {chi2:.2f}, P-value = {p_val:.3f}")
    if p_val > 0.05:
        print(f"  -> No significant difference (groups are equivalent for {col})")
    else:
        print(f"  -> Significant difference (adjust groups for {col})")

Gender: Chi-square = 8818.14, P-value = 0.000
  -> Significant difference (adjust groups for Gender)
VehicleType: Chi-square = 3336.59, P-value = 0.000
  -> Significant difference (adjust groups for VehicleType)
MaritalStatus: Chi-square = 11001.90, P-value = 0.000
  -> Significant difference (adjust groups for MaritalStatus)


In [10]:
# Add a column to indicate the group
group_a['Group'] = 'A'  # Control group
group_b['Group'] = 'B'  # Test group

# Combine into one dataset for further analysis
segmented_data = pd.concat([group_a, group_b], ignore_index=True)

# Save to a new file 
segmented_data.to_csv('segmented_data.csv', index=False)


In [11]:
# Claims Frequency
group_a_claims_frequency = group_a['TotalClaims'].notnull().sum() / len(group_a)
group_b_claims_frequency = group_b['TotalClaims'].notnull().sum() / len(group_b)

print(f"Group A Claims Frequency: {group_a_claims_frequency:.2%}")
print(f"Group B Claims Frequency: {group_b_claims_frequency:.2%}")

# Loss Ratio
group_a_loss_ratio = group_a['TotalClaims'].sum() / group_a[group_a['TotalPremium'] >0]['TotalPremium'].sum()
group_b_loss_ratio = group_b['TotalClaims'].sum() / group_b[group_b['TotalPremium'] >0]['TotalPremium'].sum()

print(f"Group A Loss Ratio: {group_a_loss_ratio:.2%}")
print(f"Group B Loss Ratio: {group_b_loss_ratio:.2%}")

Group A Claims Frequency: 100.00%
Group B Claims Frequency: 100.00%
Group A Loss Ratio: 106.45%
Group B Loss Ratio: 41.35%


In [12]:
from scipy import stats
# Hypothesis Testing for Risk Differences Across Provinces (H_1) using TotalClaims
anova_provinces = stats.f_oneway(*(df[df['Province'] == province]['TotalClaims'] for province in df['Province'].unique())) 
print("H_1 Hypothesis test result for provinces:", anova_provinces)

# Hypothesis Testing for Risk Differences Between Zip Codes (H_2) using TotalClaims
anova_zipcodes = stats.f_oneway(*(df[df['PostalCode'] == zipcode]['TotalClaims'] for zipcode in df['PostalCode'].unique()))
print("H_2 Hypothesis test result for zip codes:", anova_zipcodes)

# Hypothesis Testing for Margin Profit Differences Between Zip Codes (H_3) using TotalPremium 
anova_premium = stats.f_oneway(*(df[df['PostalCode'] == zipcode]['TotalPremium'] for zipcode in df['PostalCode'].unique())) 
print("H_3 Hypothesis test result for TotalPremium across zip codes:", anova_premium)

# Hypothesis Testing for Risk Differences Between Women and Men (H_4)
risk_women = df[df['Gender'] == 'Female']['TotalClaims']
risk_men = df[df['Gender'] == 'Male']['TotalClaims']
t_test_gender = stats.ttest_ind(risk_women, risk_men)
print("H_4 T-test result for gender:", t_test_gender)

H_1 Hypothesis test result for provinces: F_onewayResult(statistic=np.float64(5.849413762407606), pvalue=np.float64(1.6782057588675903e-07))
H_2 Hypothesis test result for zip codes: F_onewayResult(statistic=np.float64(0.9419762214391849), pvalue=np.float64(0.8906511279164051))
H_3 Hypothesis test result for TotalPremium across zip codes: F_onewayResult(statistic=np.float64(10.81111575835253), pvalue=np.float64(0.0))
H_4 T-test result for gender: TtestResult(statistic=np.float64(0.24803623812388725), pvalue=np.float64(0.8041073961270343), df=np.float64(49570.0))
